In [1]:
!pip install tinyec

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import secrets
from tinyec import registry
from Crypto.Cipher import AES

In [3]:
def compress(pubKey):
    return hex(pubKey.x)

In [4]:
curve = registry.get_curve('secp256r1')

# Public Keys

In [5]:
alicePrivKey = secrets.randbelow(curve.field.n)
alicePubKey = alicePrivKey * curve.g
print("Alice public key:", compress(alicePubKey))

Alice public key: 0xd2cfc91adb03c95aec0cbc422e04fd9dbf8bf55f6026a6186415789b68e8e748


In [6]:
bobPrivKey = secrets.randbelow(curve.field.n)
bobPubKey = bobPrivKey * curve.g
print("Bob public key:", compress(bobPubKey))

Bob public key: 0x7179ad58f368bf7643ebf051a9d6766f2603d45fd5358f74872e3370337fd033


# Selected

In [7]:
aliceSessionSecret = secrets.randbelow(curve.field.n)
print("aliceSessionSecret:", aliceSessionSecret)

aliceSelfPartialShared = aliceSessionSecret * curve.g

toBob = (alicePrivKey + aliceSessionSecret) * bobPubKey
print("\nto Bob:", compress(toBob))

aliceSessionSecret: 248210527818849416100540499604306170434430963974037113958653120129074620369

to Bob: 0xd2e07c91bbafb911b3ce8c5bb92c3434e14ea59c359db9e5eb97698086500b8f


In [8]:
bobSessionSecret = secrets.randbelow(curve.field.n)
print("bobSessionSecret:", bobSessionSecret)

bobSelfPartialShared = bobSessionSecret * curve.g

toAlice = (bobPrivKey + bobSessionSecret) * alicePubKey
print("\nto Alice:", compress(toAlice))

bobSessionSecret: 17104219107146196906665435613773405983567232540167069478074479942525261977221

to Alice: 0xdbfd6fca3553f13abe2e97b5dd97953279af1d92ec8fcad17d123e2a72a6d95d


## Computing shared

In [9]:
alicePrivKeyInv = pow(alicePrivKey, -1, curve.field.n)
alicePartialShared =  (toAlice * alicePrivKeyInv) - bobPubKey

aliceShared = aliceSelfPartialShared + alicePartialShared
print("aliceShared:", compress(aliceShared))

aliceShared: 0xa4fe7a059f4ff206222ba1c5e16f1c667fc01f8c087b7c226b3ae5ac420a0c3


In [10]:
bobPrivKeyInv = pow(bobPrivKey, -1, curve.field.n)
bobPartialShared = (toBob * bobPrivKeyInv) - alicePubKey

bobShared = bobSelfPartialShared + bobPartialShared
print("bobShared:", compress(bobShared))

bobShared: 0xa4fe7a059f4ff206222ba1c5e16f1c667fc01f8c087b7c226b3ae5ac420a0c3


In [11]:
print("Equal shared keys:", aliceShared.x == bobShared.x)

Equal shared keys: True


# Cryptography

## Encrypt

In [33]:
import base64
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad, unpad

In [77]:
class AESCipher:
    def __init__(self, intKey):
        self.key = intKey.to_bytes(32, 'big')

    def encrypt(self, msg):
        raw = str.encode(msg)
        raw = pad(raw, BS)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw)).decode("ascii")

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:BS]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt( enc[BS:] ), BS).decode()

In [78]:
cipher = AESCipher(aliceShared.x)

In [81]:
a = cipher.encrypt("sdaskdgasdlkajsdhas?AsD/asdaksdha'")
a

'Y0RHWaTOsqnIkDOqlgtrFfoFck9ZMPnMV/B2vRFBLXVPvfXRHuZR3utXnD6j+fB0ZMON/8ryS/x5ABtAbDUwFg=='

In [83]:
cipher.decrypt(a)

"sdaskdgasdlkajsdhas?AsD/asdaksdha'"

In [87]:
base64.b64encode(cipher.key).decode()

'Ck/noFn0/yBiIrocXhbxxmf8AfjAh7fCJrOuWsQgoMM='